<a href="https://colab.research.google.com/github/sedwardsmarsh/Marine-Mammal-Classifier/blob/master/Marine_Mammal_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Marine Mammal Classifier**


*   source for audio data: Watkins Marine Mammal Sound Database, Woods Hole Oceanographic Institution: https://whoicf2.whoi.edu/science/B/whalesounds/index.cfm
*   thanks to Todd Hayton for the python tutorial *Scraping by Example - Iterating through Select Items With Mechanize*: http://toddhayton.com/2015/01/09/scraping-by-example-ntu-edu/
*   this answer from stack exchange was used as well, thank you: https://stackoverflow.com/questions/5974595/download-all-the-linksrelated-documents-on-a-webpage-using-python






Before running anything, you need to tell Colab that you are interested in using a GPU. You can do this by clicking on the ‘Runtime’ tab and selecting ‘Change runtime type’. A pop-up window will open up with a drop-down menu. Select ‘GPU’ from the menu and click ‘Save’.

# ***make these images a lot smaller***

![Click the 'Runtime' tab above and select 'Change runtime type'](https://course.fast.ai/images/colab/03.png)

![A pop-up window will open up with a drop-down menu. Select ‘GPU’ from the menu and click ‘Save’.](https://course.fast.ai/images/colab/04.png)

# Setup the environment

In [25]:
# connect to google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My\ Drive/"
data_dir = root_dir + "Colab\ Notebooks/watkins_data/"

Mounted at /content/gdrive


In [26]:
# create google drive directory to hold watkins marine mammal data
!mkdir {data_dir}

mkdir: cannot create directory ‘/content/gdrive/My’: Operation not supported
mkdir: cannot create directory ‘Drive/Colab’: No such file or directory
mkdir: cannot create directory ‘Notebooks/watkins_data/’: No such file or directory


In [0]:
# fetch the latest fast.ai version 
!curl -s https://course.fast.ai/setup/colab | bash

In [0]:
# install the latest SoX version
!apt-get install -q sox

Reading package lists...
Building dependency tree...
Reading state information...
sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [1]:
# install the latest mechanize version
!pip install mechanize

     |████████████████████████████████| 112kB 4.7MB/s 


In [0]:
#!/usr/bin/env python

# import sys
import signal
import mechanize 
from time import sleep

URL = 'https://whoicf2.whoi.edu/science/B/whalesounds/index.cfm'
DELAY = 5

# def sigint(signal, frame):
#   sys.stderr.write('Exiting...\n')
#   sys.exit(0)    

class WatkinsScraper:
    def __init__(self, url=URL, delay=DELAY):
        # initilize browser, url, delay and items array
        self.br = mechanize.Browser()
        self.url = url
        self.delay = delay
        self.items = []


    def scrape(self):
        '''
        Get the list of items in the first dropdown menu, "Common name" 
        and submit the form for each item. 
        '''
        items = self.get_items()

        for item in items:
            # Skip invalid/blank item selections
            if item.get_labels != "Select":
                continue

            response = self.submit_form(item)
            links = self.get_links(filetypes=[".wav"])
            self.save_item_results(item, response, links)


    def get_items(self):
        '''
        Get the list of items in the first dropdown of the form
        '''
        self.br.open(self.url)
        self.br.select_form('jump1')

        # get items from submit tag 
        items = self.br.form.find_control('getSpeciesCommon').get_items()
        return items


    def submit_form(self, item):
        '''
        Submit form using selection item.name and download the audio files
        to data_dir
        '''
        max_tries = 3
        num_tries = 0

        while num_tries < max_tries:
            # loop through each item name from submit tag.
            try:
                self.br.open(self.url)
                self.br.select_form('jump1')
                self.br.form['getSpeciesCommon'] = [ item.name ]
                self.br.submit()
                break
            # unless encountering an error.
            except (mechanize.HTTPError, mechanize.URLError) as e:
                if isinstance(e,mechanize.HTTPError):
                    print(e.code)
                else:
                    print(e.reason.args)

            num_tries += 1
            time.sleep(num_tries * self.delay)

        if num_tries == max_tries:
            raise

        # return page response from server.
        return self.br.response().read()


    def get_links(self, filetypes=[]):
        '''
        Locates the links on a given webpage
        '''
        # myfiles holds the links of the files we want to download.
        myfiles=[]
        # iterate through links inside browser on the page.
        for link in br.links():
            # check if this link has the file extension we want.
            for ft in filetypes:
                if ft in str(link): 
                    myfiles.append(link)

        return myfiles


    def download_link(self, link):
        # perhaps you should open in a better way & ensure 
        # that file doesn't already exist.
        f = open(link.text,"w") 
        br.click_link(link)
        f.write(br.response().read())
        print ("%s has been downloaded" % link.text)


    def save_item_results(self, item, response, myfiles):
        label = ' '.join([label.text for label in item.get_labels()])
        label = '-'.join(label.split())
        
        for link in myfiles:
            # throttle so you dont hammer the site
            sleep(self.delay) 
            self.download_link(link)


if __name__ == '__main__':
    # signal.signal(signal.SIGINT, sigint)
    scraper = WatkinsScraper()
    scraper.scrape()
    # some_items = scraper.get_items()
    # token = scraper.save_item_results(item=some_items[1])
    # for x in zip(some_items): 
    #     print(x)
    